In [ ]:
#install necessary libraries for distance and dimensionality reduction tasks

IS_COLAB = True

if IS_COLAB:
    !pip install tcrdist3
    !pip install umap-learn
    !pip install umap-learn[plot]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tcrdist3: filename=tcrdist3-0.2.2-py3-none-any.whl size=2100321 sha256=26f88a0137d6124201ccd7b5bcee89e5153ab978f523d6e03220b4ad212a4444
  Stored in directory: /root/.cache/pip/wheels/da/a2/13/eea7ee0b95b1f391f5559e8bc15459e27beb2abdc59d4fce0e
  Created wheel for feather-format: filename=feather_format-0.4.1-py3-none-any.whl size=2436 sha256=bcf2934d6da10f6adb5a803a53e

In [ ]:
if IS_COLAB:
    from google.colab import drive

    drive.mount('/content/drive')
    %ls
    %cd drive/MyDrive/dsmp-2024-groupol1/

Mounted at /content/drive
drive/  sample_data/
/content/drive/MyDrive/dsmp-2024-groupol1


In [ ]:
!pip install palmotif

In [ ]:
import pandas as pd

df = pd.read_csv('data/vdjdb.txt', sep="\t")


selected_features = df[['gene','cdr3','v.segm','j.segm','species','mhc.a','mhc.b','mhc.class','antigen.epitope','antigen.species','vdjdb.score', 'complex.id']]

# Select all human data
human_data = selected_features[(selected_features['species'] == 'HomoSapiens')]

# Drop duplicate rows
human_data_cols = human_data.columns.difference(['complex.id', 'vdjdb.score'])
print(human_data_cols)
human_data = human_data.drop_duplicates()

# Delete rows with null values
human_data  = human_data.dropna()
# Print all data
human_data.head()

Index(['antigen.epitope', 'antigen.species', 'cdr3', 'gene', 'j.segm', 'mhc.a',
       'mhc.b', 'mhc.class', 'species', 'v.segm'],
      dtype='object')


,gene,cdr3,v.segm,j.segm,species,mhc.a,mhc.b,mhc.class,antigen.epitope,antigen.species,vdjdb.score,complex.id
0,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,HIV-1,2,1
1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,HIV-1,2,1
2,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,HIV-1,2,0
3,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,HIV-1,2,2
4,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,HIV-1,2,2


In [ ]:
antigen_epitope_dummies = pd.get_dummies(human_data['antigen.epitope'], dtype = int)[['ATDALMTGY', 'ELAGIGILTV', 'GILGFVFTL', 'GLCTLVAML', 'KRWIILGLNK', 'NLVPMVATV', 'RAKFKQLL']]

In [ ]:
human_data = pd.concat([human_data, antigen_epitope_dummies], axis = 1)

In [ ]:
TRB = human_data[(human_data['gene'] =='TRB') & (human_data['vdjdb.score'] > 0)]
# rename the columns for our beta chain matrix calculation
beta_chains = TRB[['cdr3', 'v.segm', 'j.segm', 'antigen.epitope', 'vdjdb.score', 'ATDALMTGY', 'ELAGIGILTV', 'GILGFVFTL', 'GLCTLVAML', 'KRWIILGLNK', 'NLVPMVATV', 'RAKFKQLL']]
beta_chains.rename(columns={'cdr3':'cdr3_b_aa','v.segm':'v_b_gene', 'j.segm':'j_b_gene'}, inplace=True)
beta_chains

<ipython-input-94-07ae6c494c60>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beta_chains.rename(columns={'cdr3':'cdr3_b_aa','v.segm':'v_b_gene', 'j.segm':'j_b_gene'}, inplace=True)


,cdr3_b_aa,v_b_gene,j_b_gene,antigen.epitope,vdjdb.score,ATDALMTGY,ELAGIGILTV,GILGFVFTL,GLCTLVAML,KRWIILGLNK,NLVPMVATV,RAKFKQLL
1,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,FLKEKGGL,2,0,0,0,0,0,0,0
2,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,FLKEKGGL,2,0,0,0,0,0,0,0
4,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,FLKEKGGL,2,0,0,0,0,0,0,0
6,CASSYEPGQVSHYSNQPQHF,TRBV13*01,TRBJ1-5*01,FLKEKGGL,2,0,0,0,0,0,0,0
8,CASSALASLNEQFF,TRBV14*01,TRBJ2-1*01,FLKEKGGL,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
92682,CASSLRATDTQYF,TRBV7-2*01,TRBJ2-3*01,PQPELPYPQPQL,2,0,0,0,0,0,0,0
92684,CATSRAGGGGEKLFF,TRBV15*01,TRBJ1-4*01,FPQPEQPFPWQP,2,0,0,0,0,0,0,0
92686,CASSQGSGGNEQFF,TRBV4-3*01,TRBJ2-1*01,FPQPEQPFPWQP,2,0,0,0,0,0,0,0
92690,CASSIVGSGGYNEQFF,TRBV19*01,TRBJ2-1*01,QLQPFPQPELPY,2,0,0,0,0,0,0,0


In [ ]:
beta_chains['antigen.epitope'].astype('category').cat.codes

1         77
2         77
4         77
6         77
8         77
        ... 
92682    258
92684     88
92686     88
92690    271
92768    260
Length: 4698, dtype: int16

In [ ]:
from sklearn.model_selection import train_test_split
from tcrdist.repertoire import TCRrep

import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.metrics import accuracy_score, classification_report, f1_score, pairwise
from sklearn.decomposition import PCA


np.random.seed(1209)

def build_model(X_train, X_test, y_train, y_test):


  hyparams = {'metric': 'euclidean', 'n_neighbors': 20, 'weights': 'distance'}
  knn = KNeighborsClassifier(**hyparams)

  knn.fit(X_train, y_train)

  # Display the best parameters and the score of the best model
  #best_knn = grid_search.best_estimator_
  y_pred = knn.predict(X_test)
  print(classification_report(y_pred, y_test))
  print(f1_score(y_pred, y_test, average = 'macro'))

  nn = NearestNeighbors(n_neighbors=20, metric='euclidean')
  nn.fit(X_train)

  # distances and indices of the 20 nearest neighbors in the training set
  distances, indices = nn.kneighbors(X_test)
  return distances, indices, y_pred


def process_and_evaluate(df, chain):
  #value_counts_antigen = df['antigen.epitope'].value_counts()
  if chain == ['alpha', 'beta']:
    list_of_eps = ['ATDALMTGY', 'ELAGIGILTV', 'GILGFVFTL', 'GLCTLVAML',  'NLVPMVATV', 'RAKFKQLL']
  else:
    list_of_eps = ['ATDALMTGY', 'ELAGIGILTV', 'GILGFVFTL', 'GLCTLVAML', 'KRWIILGLNK', 'NLVPMVATV', 'RAKFKQLL']

  matrix_filtered = df[df['antigen.epitope'].isin(list_of_eps)]
  matrix_filtered['antigen.epitope.cat'] = matrix_filtered['antigen.epitope'].astype('category').cat.codes
  train_x, test_x = train_test_split(matrix_filtered, test_size = 0.2, stratify = matrix_filtered['antigen.epitope'])

  print(test_x['antigen.epitope'].value_counts())
  print(test_x['antigen.epitope.cat'].value_counts())
  tr = TCRrep(cell_df = train_x,
            organism = 'human',
            chains = chain,
            db_file = 'alphabeta_gammadelta_db.tsv')

  test_tcrs = TCRrep(cell_df=test_x,
                   organism='human',
                   chains=chain,
                   deduplicate=True)

  test_tcrs.compute_rect_distances(df=test_tcrs.clone_df,
                                    df2=tr.clone_df)

  if (len(chain)==1) and (chain[0] == 'beta'):
    distance_matrix = pd.DataFrame(tr.pw_cdr3_b_aa)
    distance_matrix_test = pd.DataFrame(test_tcrs.rw_cdr3_b_aa)
    chains = 'beta'
  elif (len(chain)==1) and (chain[0] == 'alpha'):
    distance_matrix = pd.DataFrame(tr.pw_cdr3_a_aa)
    distance_matrix_test = pd.DataFrame(test_tcrs.rw_cdr3_a_aa)
    chains = 'alpha'
  else:
    print('Hey')
    distance_matrix = pd.DataFrame(np.hstack((tr.pw_cdr3_b_aa, tr.pw_cdr3_a_aa)))
    distance_matrix_test = pd.DataFrame(np.hstack((test_tcrs.rw_cdr3_b_aa, test_tcrs.rw_cdr3_a_aa)))
    chains = 'alpha_beta'

  y_train = tr.clone_df['antigen.epitope.cat']
  y_test = test_tcrs.clone_df['antigen.epitope.cat']

  #print(distance_matrix_test.shape)

  #print('X train shape, ', distance_matrix.shape)
  #scaler = StandardScaler()
  #scaler.fit(distance_matrix)
  X_train = distance_matrix
  X_test = distance_matrix_test
  PCA_model = PCA(n_components = 30)
  X_train = PCA_model.fit_transform(X_train)

  #
  X_test = PCA_model.transform(X_test)
  print('X train shape, ', X_train.shape)
  print('X test shape, ', X_test.shape)
  distances, indices, y_pred = build_model(X_train, X_test, y_train, y_test)
  return distances, indices, y_pred, y_test, test_tcrs.clone_df['antigen.epitope']



distances, indices, pred, test, epitope_name = process_and_evaluate(beta_chains, ['beta'])


<ipython-input-104-a51cc5e0ce37>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matrix_filtered['antigen.epitope.cat'] = matrix_filtered['antigen.epitope'].astype('category').cat.codes
/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")


antigen.epitope
NLVPMVATV     129
GILGFVFTL      72
GLCTLVAML      47
RAKFKQLL       33
ELAGIGILTV     30
KRWIILGLNK     29
ATDALMTGY      28
Name: count, dtype: int64
antigen.epitope.cat
5    129
2     72
3     47
6     33
1     30
4     29
0     28
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")


X train shape,  (1217, 30)
X test shape,  (349, 30)
              precision    recall  f1-score   support

           0       0.71      0.69      0.70        29
           1       0.30      0.69      0.42        13
           2       0.78      0.85      0.81        54
           3       0.67      0.82      0.74        38
           4       0.45      0.46      0.46        28
           5       0.90      0.65      0.75       173
           6       0.24      0.57      0.34        14

    accuracy                           0.68       349
   macro avg       0.58      0.68      0.60       349
weighted avg       0.76      0.68      0.70       349

0.6033411720370258


In [ ]:
analyse_df = pd.concat((pd.DataFrame(distances), pd.Series(pred, name = 'predictions'), test, epitope_name), axis = 1)

In [ ]:
analyse_df.columns

Index([                    0,                     1,                     2,
                           3,                     4,                     5,
                           6,                     7,                     8,
                           9,                    10,                    11,
                          12,                    13,                    14,
                          15,                    16,                    17,
                          18,                    19,         'predictions',
       'antigen.epitope.cat',     'antigen.epitope'],
      dtype='object')

In [ ]:
analyse_df['Correct'] = analyse_df.apply(lambda x: True if x['predictions'] == x['antigen.epitope.cat'] else False, axis = 1)

In [ ]:
analyse_df['Mean Distance'] = analyse_df.iloc[:, :-4].apply(np.mean, axis = 1)
analyse_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,predictions,antigen.epitope.cat,antigen.epitope,Correct,Mean Distance
0,0.232204,86.354605,93.984101,94.589683,102.839256,105.117285,115.362192,117.969012,118.620987,119.262578,...,141.838206,144.543859,145.482299,145.879302,148.463930,5,5,NLVPMVATV,True,110.790228
1,55.550595,88.736292,100.955181,102.039224,107.458025,107.890759,117.696587,129.097764,135.063715,135.731176,...,148.123445,148.917725,151.877229,152.991296,153.906004,3,3,GLCTLVAML,True,120.701491
2,103.892450,108.416669,111.117245,117.406891,126.431344,127.829250,130.871652,135.000543,138.103282,142.835416,...,150.063771,151.556503,152.517379,158.109500,159.714708,5,0,ATDALMTGY,False,130.964106
3,76.731466,89.177611,105.696379,108.410855,118.047226,135.085793,140.802662,141.287732,146.975500,149.236086,...,163.029008,165.275881,167.328262,170.714187,171.579848,5,0,ATDALMTGY,False,135.112833
4,24.138784,69.658627,70.581668,98.067197,102.803253,102.803253,107.786540,111.079421,113.996241,114.489082,...,120.884931,121.785447,122.196136,122.307245,123.116596,0,0,ATDALMTGY,True,100.626293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0.044013,0.044013,0.044013,9.209993,21.619383,26.761593,26.761593,33.672443,42.201029,68.212468,...,71.964423,110.928978,116.509181,117.561963,137.403922,3,3,GLCTLVAML,True,54.164781
345,91.582870,93.919929,103.170672,107.207422,108.945274,113.659027,116.119557,118.246468,118.390953,121.336188,...,128.353687,128.492494,130.854273,133.350998,135.912482,4,6,RAKFKQLL,False,113.230845
346,77.287182,83.088672,91.893012,106.162793,107.235702,110.837909,111.983049,117.598077,121.951867,125.745482,...,133.328323,133.925710,133.925710,140.744240,142.653157,6,3,GLCTLVAML,False,113.788327
347,53.855354,80.314576,82.297181,92.327745,104.446872,105.122561,107.084170,108.362610,111.523179,113.878079,...,120.540876,122.040656,122.162119,122.543987,122.779701,3,6,RAKFKQLL,False,103.048973


In [ ]:
analyse_df.groupby(['antigen.epitope', 'Correct'])['Mean Distance'].mean()

antigen.epitope  Correct
ATDALMTGY        False      122.984068
                 True       105.653072
ELAGIGILTV       False      120.492909
                 True       104.393861
GILGFVFTL        False      127.069858
                 True        62.608256
GLCTLVAML        False      128.272948
                 True        88.874836
KRWIILGLNK       False      121.730771
                 True       100.597002
NLVPMVATV        False      140.569680
                 True       108.313919
RAKFKQLL         False      119.335437
                 True       112.523508
Name: Mean Distance, dtype: float64

In [ ]:
analyse_df.groupby(['antigen.epitope'])['Mean Distance'].mean()

antigen.epitope
ATDALMTGY     110.604785
ELAGIGILTV    115.663195
GILGFVFTL      76.811660
GLCTLVAML     101.722046
KRWIILGLNK    112.257013
NLVPMVATV     111.435444
RAKFKQLL      117.684060
Name: Mean Distance, dtype: float64

In [ ]:
np.random.seed(1209)
TRA = human_data[(human_data['gene'] =='TRA') & (human_data['vdjdb.score'] > 0)]
# rename the columns for our beta chain matrix calculation
alpha_chains = TRA[['cdr3', 'v.segm', 'j.segm', 'antigen.epitope', 'vdjdb.score', 'ATDALMTGY', 'ELAGIGILTV', 'GILGFVFTL', 'GLCTLVAML', 'KRWIILGLNK', 'NLVPMVATV', 'RAKFKQLL']]
alpha_chains.rename(columns={'cdr3':'cdr3_a_aa','v.segm':'v_a_gene', 'j.segm':'j_a_gene'}, inplace=True)
alpha_chains
_ = process_and_evaluate(alpha_chains, ['alpha'])

<ipython-input-105-65dd2b4525d4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alpha_chains.rename(columns={'cdr3':'cdr3_a_aa','v.segm':'v_a_gene', 'j.segm':'j_a_gene'}, inplace=True)
<ipython-input-104-a51cc5e0ce37>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matrix_filtered['antigen.epitope.cat'] = matrix_filtered['antigen.epitope'].astype('category').cat.codes
/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/usr/local/lib/pytho

antigen.epitope
NLVPMVATV     93
GILGFVFTL     53
GLCTLVAML     17
ELAGIGILTV     5
RAKFKQLL       1
KRWIILGLNK     1
Name: count, dtype: int64
antigen.epitope.cat
4    93
1    53
2    17
0     5
5     1
3     1
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")


X train shape,  (438, 30)
X test shape,  (147, 30)
              precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       0.69      0.90      0.78        30
           2       0.64      0.82      0.72        11
           3       0.00      0.00      0.00         0
           4       0.98      0.83      0.90       102
           5       1.00      1.00      1.00         1

    accuracy                           0.85       147
   macro avg       0.65      0.76      0.69       147
weighted avg       0.89      0.85      0.86       147

0.6920132658538455


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
np.random.seed(1209)

alpha_chain = human_data[(human_data['gene'] == 'TRA') & (human_data['complex.id'] != 0)]
beta_chain = human_data[(human_data['gene'] == 'TRB') & (human_data['complex.id'] != 0)]

alpha_chain.rename(columns = {'cdr3':'cdr3_a_aa','v.segm':'v_a_gene', 'j.segm':'j_a_gene'}, inplace = True)
beta_chain.rename(columns = {'cdr3':'cdr3_b_aa','v.segm':'v_b_gene', 'j.segm':'j_b_gene'}, inplace = True)

combined_chain = pd.merge(alpha_chain[['cdr3_a_aa', 'v_a_gene', 'j_a_gene', 'mhc.a', 'antigen.epitope', 'vdjdb.score', 'complex.id']].reset_index(drop = True),
                          beta_chain[['cdr3_b_aa', 'v_b_gene', 'j_b_gene', 'mhc.b', 'complex.id']].reset_index(drop = True),
                          how = 'inner',
                          on = 'complex.id')

_ = process_and_evaluate(combined_chain, ['alpha', 'beta'])

<ipython-input-103-5dfff0bffcab>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alpha_chain.rename(columns = {'cdr3':'cdr3_a_aa','v.segm':'v_a_gene', 'j.segm':'j_a_gene'}, inplace = True)
<ipython-input-103-5dfff0bffcab>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  beta_chain.rename(columns = {'cdr3':'cdr3_b_aa','v.segm':'v_b_gene', 'j.segm':'j_b_gene'}, inplace = True)
<ipython-input-95-a51cc5e0ce37>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

antigen.epitope
GILGFVFTL     432
RAKFKQLL      245
NLVPMVATV     111
ELAGIGILTV     78
GLCTLVAML      77
Name: count, dtype: int64
antigen.epitope.cat
1    432
4    245
3    111
0     78
2     77
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")


Hey
X train shape,  (3772, 30)
X test shape,  (943, 30)
              precision    recall  f1-score   support

           0       0.33      0.62      0.43        42
           1       0.86      0.83      0.84       448
           2       0.65      0.68      0.67        73
           3       0.71      0.59      0.64       135
           4       0.65      0.65      0.65       245

    accuracy                           0.73       943
   macro avg       0.64      0.67      0.65       943
weighted avg       0.74      0.73      0.73       943

0.6468875665565561


In [ ]:
#dummy_preproc = beta_chains.groupby('cdr3_b_aa')[['ATDALMTGY', 'ELAGIGILTV', 'GILGFVFTL', 'GLCTLVAML', 'KRWIILGLNK', 'NLVPMVATV', 'RAKFKQLL']].sum().reset_index()
#beta_chains_proc = pd.merge(beta_chains[['cdr3_b_aa',	'v_b_gene',	'j_b_gene',	'antigen.epitope']], dummy_preproc, how = 'left', on = 'cdr3_b_aa').drop_duplicates(subset = ['cdr3_b_aa'])
#beta_chains_proc.reset_index(inplace = True)

In [ ]:
beta_chains_proc = beta_chains

In [ ]:
condition = (
    (beta_chains_proc['ATDALMTGY'] == 0) &
    (beta_chains_proc['ELAGIGILTV'] == 0) &
    (beta_chains_proc['GILGFVFTL'] == 0) &
    (beta_chains_proc['GLCTLVAML'] == 0) &
    (beta_chains_proc['KRWIILGLNK'] == 0) &
    (beta_chains_proc['NLVPMVATV'] == 0) &
    (beta_chains_proc['RAKFKQLL'] == 0)
)

# Drop the rows where the condition is True
beta_chains_proc = beta_chains_proc[~condition]
beta_chains_proc.shape

(12573, 12)

In [ ]:
beta_chains_proc['vdjdb.score'].value_counts()

vdjdb.score
0    11175
1      747
2      390
3      261
Name: count, dtype: int64

In [ ]:
import numpy as np

filtered_indices = beta_chains_proc[beta_chains_proc['vdjdb.score']==0].index
selected_indices = np.random.choice(filtered_indices, size=9500, replace=False)

beta_chains_proc = beta_chains_proc.drop(selected_indices)
beta_chains_proc.shape

(3073, 12)

In [ ]:
beta_chains_proc['classifer'] = beta_chains_proc['vdjdb.score'].apply(lambda x: 1 if x>0 else 0)

In [ ]:
beta_chains_proc

,cdr3_b_aa,v_b_gene,j_b_gene,antigen.epitope,vdjdb.score,ATDALMTGY,ELAGIGILTV,GILGFVFTL,GLCTLVAML,KRWIILGLNK,NLVPMVATV,RAKFKQLL,classifer
683,CASSPQGLGTEAFF,TRBV28*01,TRBJ1-1*01,ELAGIGILTV,1,0,1,0,0,0,0,0,1
703,CASSQGLAGAGELFF,TRBV19*01,TRBJ2-2*01,ELAGIGILTV,0,0,1,0,0,0,0,0,0
709,CASSYSATGGEQYF,TRBV6-5*01,TRBJ2-7*01,ELAGIGILTV,0,0,1,0,0,0,0,0,0
724,CASSYGGLGQPQHF,TRBV27*01,TRBJ1-5*01,ELAGIGILTV,0,0,1,0,0,0,0,0,0
743,CASSPSQGGNTEAFF,TRBV27*01,TRBJ1-1*01,ELAGIGILTV,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89516,CATSDVNGAYEQYF,TRBV24-1*01,TRBJ2-7*01,KRWIILGLNK,1,0,0,0,0,1,0,0,1
90114,CASSPGQGEGYEQYF,TRBV4-1*01,TRBJ2-7*01,RAKFKQLL,0,0,0,0,0,0,0,1,0
90117,CASSDLNSPLHF,TRBV27*01,TRBJ1-6*01,RAKFKQLL,0,0,0,0,0,0,0,1,0
90123,CSARDRGTENTGELFF,TRBV20-1*01,TRBJ2-2*01,RAKFKQLL,0,0,0,0,0,0,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(beta_chains_proc, test_size = 0.2, stratify = beta_chains_proc['classifer'])

In [ ]:
X_train

,cdr3_b_aa,v_b_gene,j_b_gene,antigen.epitope,vdjdb.score,ATDALMTGY,ELAGIGILTV,GILGFVFTL,GLCTLVAML,KRWIILGLNK,NLVPMVATV,RAKFKQLL,classifer
72757,CATSDGLAGGWANVLTF,TRBV24-1*01,TRBJ2-6*01,GILGFVFTL,1,0,0,1,0,0,0,0,1
17336,CASSLIYPGELFF,TRBV27*01,TRBJ2-2*01,GILGFVFTL,3,0,0,1,0,0,0,0,1
16942,CASSRRSADEQFF,TRBV19*01,TRBJ2-1*01,GILGFVFTL,0,0,0,1,0,0,0,0,0
35347,CASSVRSSDTQYF,TRBV19*01,TRBJ2-3*01,GILGFVFTL,0,0,0,1,0,0,0,0,0
18598,CASSLGGGGGNQPQHF,TRBV7-8*01,TRBJ1-5*01,GLCTLVAML,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11682,CASSLVAGQIITDTQYF,TRBV5-6*01,TRBJ2-3*01,NLVPMVATV,0,0,0,0,0,0,1,0,0
11726,CASSIRTGQTGELFF,TRBV28*01,TRBJ2-2*01,NLVPMVATV,0,0,0,0,0,0,1,0,0
981,CAISHTGELFF,TRBV10-3*01,TRBJ2-2*01,KRWIILGLNK,1,0,0,0,0,1,0,0,1
26210,CASSRYHTDTQYF,TRBV19*01,TRBJ2-3*01,GILGFVFTL,1,0,0,1,0,0,0,0,1


In [ ]:
from sklearn.decomposition import PCA
from tcrdist.repertoire import TCRrep

tr = TCRrep(cell_df = X_train,
            organism = 'human',
            chains = ['beta'],
            db_file = 'alphabeta_gammadelta_db.tsv')

tr

/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")


In [ ]:
from sklearn.preprocessing import StandardScaler

df_train = pd.DataFrame(tr.pw_cdr3_b_aa)
scaler = StandardScaler()
scaler.fit(df_train)
df_train = scaler.transform(df_train)
PCA_model = PCA(n_components = 30)
reduced_pca = PCA_model.fit_transform(df_train)
#, tr.clone_df[['ATDALMTGY', 'ELAGIGILTV', 'GILGFVFTL', 'GLCTLVAML', 'KRWIILGLNK', 'NLVPMVATV', 'RAKFKQLL']]], axis = 1)

In [ ]:
X_train_scaled = np.hstack((reduced_pca,
                           tr.clone_df[['ATDALMTGY', 'ELAGIGILTV', 'GILGFVFTL', 'GLCTLVAML', 'KRWIILGLNK', 'NLVPMVATV', 'RAKFKQLL']].values))

X_train_scaled.shape

(2448, 37)

In [ ]:
y_train = tr.clone_df['classifer']
y_train.shape

(2448,)

In [ ]:
test_tcrs = TCRrep(cell_df=X_test,
                   organism='human',
                   chains=['beta'],
                   deduplicate=True)

test_tcrs.compute_rect_distances(df=test_tcrs.clone_df,
                                    df2=tr.clone_df)

/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:159: UserWarning: cell_df needs a counts column to track clonal number of frequency

  self._validate_cell_df()
/usr/local/lib/python3.10/dist-packages/tcrdist/repertoire.py:791: UserWarning: No 'count' column provided; count column set to 1
  warnings.warn("No 'count' column provided; count column set to 1")


In [ ]:
df_test = scaler.transform(test_tcrs.rw_cdr3_b_aa)
PCA_model = PCA(n_components = 30)
reduced_pca_test = PCA_model.fit_transform(df_test)
X_test_scaled = np.hstack((reduced_pca_test,
                           test_tcrs.clone_df[['ATDALMTGY', 'ELAGIGILTV', 'GILGFVFTL', 'GLCTLVAML', 'KRWIILGLNK', 'NLVPMVATV', 'RAKFKQLL']].values))

In [ ]:
y_test = test_tcrs.clone_df['classifer']

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

param_grid = {
    'n_neighbors': [3, 5, 7, 10, 15, 20, 25, 30],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean']
}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, verbose=1)

# Fit the grid search to the data
grid_search.fit(X_train_scaled, y_train)

# Display the best parameters and the score of the best model
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test_scaled)

accuracy_score(y_pred, y_test)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters: {'metric': 'euclidean', 'n_neighbors': 15, 'weights': 'distance'}
Best score: 0.5686473853345019


0.5691056910569106

In [ ]:
print(y_pred.shape)
print(y_test.shape)
print(test_tcrs.clone_df['antigen.epitope'].shape)
eval_df = pd.concat([pd.Series(y_pred), y_test], axis = 1)
eval_df.rename(columns = {0: 'Predicted', 'classifer': 'Actual'}, inplace = True)

eval_df['Epitope'] = test_tcrs.clone_df['antigen.epitope']
eval_df

(615,)
(615,)
(615,)


,Predicted,Actual,Epitope
0,0,0,RAKFKQLL
1,0,0,ELAGIGILTV
2,0,0,NLVPMVATV
3,0,1,ATDALMTGY
4,0,1,ATDALMTGY
...,...,...,...
610,0,1,NLVPMVATV
611,0,0,NLVPMVATV
612,1,0,GILGFVFTL
613,0,0,NLVPMVATV


In [ ]:
eval_df['Correct'] = eval_df.apply(lambda x: 'Correct' if x['Predicted'] == x ['Actual'] else 'Incorrect', axis = 1)
eval_df.groupby('Epitope')['Correct'].value_counts()

Epitope     Correct  
ATDALMTGY   Incorrect     18
            Correct       13
ELAGIGILTV  Correct       42
            Incorrect     29
GILGFVFTL   Correct       85
            Incorrect     47
GLCTLVAML   Correct       41
            Incorrect     23
KRWIILGLNK  Incorrect     31
            Correct       12
NLVPMVATV   Correct      129
            Incorrect     90
RAKFKQLL    Correct       28
            Incorrect     27
Name: count, dtype: int64

In [ ]:
eval_df.groupby('Actual')['Correct'].value_counts()

Actual  Correct  
0       Correct      240
        Incorrect     95
1       Incorrect    170
        Correct      110
Name: count, dtype: int64

In [ ]:
eval_df['VDJD_SCORE'] = test_tcrs.clone_df['vdjdb.score']
eval_df

,Predicted,Actual,Epitope,Correct,VDJD_SCORE
0,0,0,RAKFKQLL,Correct,0
1,0,0,ELAGIGILTV,Correct,0
2,0,0,NLVPMVATV,Correct,0
3,0,1,ATDALMTGY,Incorrect,1
4,0,1,ATDALMTGY,Incorrect,1
...,...,...,...,...,...
610,0,1,NLVPMVATV,Incorrect,3
611,0,0,NLVPMVATV,Correct,0
612,1,0,GILGFVFTL,Incorrect,0
613,0,0,NLVPMVATV,Correct,0


In [ ]:
eval_df.groupby('VDJD_SCORE')['Correct'].value_counts()

VDJD_SCORE  Correct  
0           Correct      240
            Incorrect     95
1           Incorrect    100
            Correct       52
2           Incorrect     39
            Correct       38
3           Incorrect     31
            Correct       20
Name: count, dtype: int64